In [1]:
# necessary packages
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
# APC/AVL and OCRA files to merge
filepath = 'C:/Users/mstark/Desktop/DATA591/'

In [3]:
# winter data
orca_file = filepath + 'orcadata_winter.csv'
avl_file = filepath + 'apcdata_winter.csv'
days_to_keep = [f'2019-01-{day:02d}' for day in range(7, 32)] +  \
               [f'2019-02-{day:02d}' for day in range(1, 3)] +  \
               [f'2019-02-{day:02d}' for day in range(13, 29)] + \
               [f'2019-03-{day:02d}' for day in range(1, 4)]
            
# summer data
# orca_file = filepath + 'orcadata_summer.csv'
# avl_file = filepath + 'apcdata_summer.csv'
# days_to_keep = [f'2019-07-{day:02d}' for day in range(1, 32)]  + \
#                [f'2019-08-{day:02d}' for day in range(1, 32)]

In [4]:
# read in raw ORCA transactions
orca_data = pd.read_csv(orca_file)
print('Rows, Features:', orca_data.shape)
print('Columns:', list(orca_data.columns))
o_0 = orca_data.shape[0]

C:\Users\mstark\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (26,36,37,40,41,46,47,48,53) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Rows, Features: (27878358, 61)
Columns: ['txn_id', 'trip_group_id', 'prev_leg_txn_id', 'card_serial_number', 'institution_id', 'institution_name', 'business_date', 'txn_dtm_pacific', 'txn_type_id', 'txn_subtype_id', 'txn_type_descr', 'upgrade_indicator', 'product_id', 'product_descr', 'txn_passenger_type_id', 'txn_passenger_type_descr', 'passenger_count', 'ceffv', 'service_agency_id', 'service_agency_name', 'source_agency_id', 'source_agency_name', 'transit_operator_abbrev', 'mode_id', 'mode_abbrev', 'mode_descr', 'route_number', 'direction_id', 'direction_descr', 'agency_trip_id', 'device_id', 'device_type', 'device_place_name', 'device_place_id', 'device_location_id', 'device_location_code', 'device_location_abbrev', 'device_location_descr', 'origin_location_id', 'origin_location_code', 'origin_location_abbrev', 'origin_location_descr', 'destination_location_id', 'destination_location_code', 'destination_location_abbrev', 'destination_location_descr', 'device_id_filt', 'stop_id', 'st

In [5]:
# print any duplicate in OCRA data
ocra_dups = orca_data[orca_data.duplicated()]
print('Duplicate records:', ocra_dups.shape[0])
ocra_dups

Duplicate records: 0


,txn_id,trip_group_id,prev_leg_txn_id,card_serial_number,institution_id,institution_name,business_date,txn_dtm_pacific,txn_type_id,txn_subtype_id,...,stop_error,viaserviceareaid,viaserviceareaname,trip_id,last_mode_id,last_route_number,last_stop_id,last_stop_time,last_stop_lat,last_stop_lon


In [6]:
orca_data['route_number']

0           245
1           510
2            40
3             3
4           676
5           522
6           542
7           560
8            33
9           542
10            3
11          542
12          672
13          271
14          167
15          271
16          672
17          550
18            4
19          550
20          532
21          532
22          130
23          512
24           33
25          554
26           70
27          542
28           24
29            3
           ... 
27878328    109
27878329    421
27878330      2
27878331     70
27878332    240
27878333     40
27878334     11
27878335     70
27878336    522
27878337     45
27878338      2
27878339    594
27878340    512
27878341    512
27878342    532
27878343    412
27878344     29
27878345     11
27878346    550
27878347     32
27878348    512
27878349     33
27878350     36
27878351    550
27878352    550
27878353    554
27878354    240
27878355      3
27878356     70
27878357    855
Name: route_number, Leng

In [7]:
# reduce columns
orca_cols_to_keep = ['institution_name', 'business_date', 'txn_dtm_pacific', 'txn_passenger_type_descr',
                     'passenger_count', 'service_agency_id', 'mode_id', 'route_number', 'direction_descr',
                     'direction_id', 'stop_id', 'trip_id']
orca_data = orca_data[orca_cols_to_keep]
o_1 = orca_data.shape[0]

In [8]:
# keep days, service_agency_id = 4, mode_id in [128, 250], and route_numbers [<600, between 671 and 676]
orca_data['route_number'] = pd.to_numeric(orca_data['route_number'], errors='coerce')
orca_data = orca_data[(orca_data['business_date'].isin(days_to_keep)) 
                      & (orca_data['service_agency_id'] == 4)
                      & (orca_data['mode_id'].isin([128, 250]))
                      & ((orca_data['route_number'] < 600) 
                         | ((orca_data['route_number'] >= 671) & (orca_data['route_number'] <= 676)))]
o_2 = orca_data.shape[0]

In [9]:
# add features

# orca_data['is_rapidride'] = [1 if x > 600 else 0 for x in orca_data['route_number']]
# orca_data['day_of_week'] = pd.to_datetime(orca_data['business_date']).dt.dayofweek
# orca_data['biz_txn_diff'] = (pd.to_datetime(orca_data['business_date']).dt.date 
#                              - pd.to_datetime(orca_data['txn_dtm_pacific']).dt.date)/np.timedelta64(1, 'D')

orca_data['orca_total'] = orca_data['passenger_count']
orca_data['orca_adult'] = orca_data['passenger_count'].where(orca_data['txn_passenger_type_descr'] == 'Adult', 0)
orca_data['orca_disabled'] = orca_data['passenger_count'].where(orca_data['txn_passenger_type_descr'] == 'Disabled', 0)
orca_data['orca_senior'] = orca_data['passenger_count'].where(orca_data['txn_passenger_type_descr'] == 'Senior', 0)
orca_data['orca_youth'] = orca_data['passenger_count'].where(orca_data['txn_passenger_type_descr'] == 'Youth', 0)
orca_data['orca_lowincome'] = orca_data['passenger_count'].where(orca_data['txn_passenger_type_descr'] == 'Low Income', 0)
orca_data['orca_uw'] = orca_data['passenger_count'].where(orca_data['institution_name'] == 'University of Washington', 0)

In [10]:
# cleaned ORCA results
print('Preserved', o_2, 'rows of', o_0, 'from OCRA transtions.', o_2*1.0/o_0*100, '%')

Preserved 9788082 rows of 27878358 from OCRA transtions. 35.10996594562707 %


In [11]:
# aggregate ORCA over day, trip, stop, route
orca_agg_groupby = ['business_date', 'trip_id', 'stop_id', 'route_number']
orca_agg_sumover = ['orca_total', 'orca_adult', 'orca_disabled', 'orca_senior', 'orca_youth', 'orca_lowincome', 'orca_uw']
orca_agg = orca_data[orca_agg_groupby + orca_agg_sumover].groupby(orca_agg_groupby).sum().reset_index()
print('Rows:', orca_agg.shape[0])

Rows: 4792868


In [13]:
# read in AVL/APC data
avl_data = pd.read_csv(avl_file)
print('Rows, Features:', avl_data.shape)
a_0 = avl_data.shape[0]

Rows, Features: (20535948, 32)


In [14]:
avl_data.head()

,Unnamed: 0,daycode,trip_id,pattern_id,pattern_quality,blk,rte,dir,sch_st_min,opd_date,...,apc_veh,ons,offs,load,geom,sch_stop_tm.1,act_stop_tm.1,stop_datetime,gps_lat,gps_long
0,0,0,40956049,10994010,100,99401,994,N,467.0,2019-01-07,...,Y,0.0,7.0,0.0,ST,NaN,NaN,2019-01-07 07:49:57,NaN,0101000020E61000002D15D1C50B955EC0BD6EE4D599DD...
1,1,0,40956861,82565531,100,99402,994,S,994.0,2019-01-07,...,N,NaN,NaN,NaN,STE,NaN,NaN,2019-01-07 16:40:17,NaN,0101000020E61000004C9BD4951D955EC044C3F693B8CC...
2,2,0,40421104,40989003,100,98902,989,E,1002.0,2019-01-07,...,Y,0.0,1.0,0.0,STE,NaN,NaN,2019-01-07 16:14:36,NaN,0101000020E610000074BDADDFF58A5EC05720E56F4DC8...
3,3,0,40422735,82565529,100,98901,989,W,488.0,2019-01-07,...,Y,0.0,17.0,0.0,ST,NaN,NaN,2019-01-07 08:10:10,NaN,0101000020E61000006968101067955EC05A1F426864DE...
4,4,0,40956859,82565528,100,98802,988,S,991.0,2019-01-07,...,N,NaN,NaN,NaN,STE,NaN,NaN,2019-01-07 16:25:08,NaN,0101000020E61000004839443162935EC0BCDDF707C8CD...


In [ ]:
avl_header = ['daycode', 'trip_id', 'pattern_id', 'pattern_quality', 'blk', 'rte', 'dir', 'sch_st_min', 'opd_date',
              'pattern_quality_1', 'vehicle_id', 'stop_id', 'stop_seq', 'stop_name', 'sch_stop_sec', 'act_stop_arr',
              'sch_stop_tm', 'act_stop_tm', 'dwell_sec', 'doors_open', 'door_open_sec', 'apc_veh', 'ons', 'offs',
              'load', 'geom', 'sch_stop_tm', 'act_stop_tm', 'stop_datetime','gps_lat', 'gps_long']
avl_data.columns = avl_header

In [ ]:
# check duplicates in raw file
avl_dups = avl_data[avl_data.duplicated()]
a_05 = avl_dups.shape[0]
print('Duplicate records:', avl_dups.shape[0])
avl_dups.head(10)

In [ ]:
# drop duplicate records
avl_data = avl_data.drop_duplicates()
a_1 = avl_data.shape[0]
print('Validate all rows accounted for:', a_0, a_05+a_1)

In [ ]:
# keep days, apc vehic le, and rte [<600, between 671 and 676]
avl_data = avl_data[(avl_data['opd_date'].isin(days_to_keep))
                   & (avl_data['apc_veh'] == 'Y')
                   & ((avl_data['rte'].astype(int) < 600) 
                      | ((avl_data['rte'].astype(int) >= 671) & (avl_data['rte'].astype(int) <= 676)))]
a_2 = avl_data.shape[0]

In [ ]:
# add features
avl_data['day_of_week'] = pd.to_datetime(avl_data['opd_date']).dt.dayofweek
avl_data['is_rapidride'] = [1 if x > 600 else 0 for x in avl_data['rte'].astype(int)]
avl_data['opd_txn_diff'] = (pd.to_datetime(avl_data['opd_date']).dt.date
                             - pd.to_datetime(avl_data['stop_datetime']).dt.date)/np.timedelta64(1, 'D')
avl_data['ons_update'] = [x if x < 150 else None for x in avl_data['ons']]

In [ ]:
# cleaned APC results
print('Preserved', a_2, 'row of', a_0, 'from APC transtions.', a_2*1.0/a_0*100, '%')

In [ ]:
# aggregate APC over day, trip, stop, route
avl_agg_groupby = ['opd_date', 'trip_id', 'stop_id', 'rte', 'dir', 'is_rapidride', 'day_of_week']
avl_agg_sumover = ['ons', 'offs', 'load', 'ons_update']
avl_agg = avl_data[avl_agg_groupby + avl_agg_sumover].groupby(avl_agg_groupby).sum().reset_index()
print('Rows:', avl_agg.shape[0])

In [ ]:
# merge orca_agg and apc_agg
merged_data = pd.merge(avl_agg, orca_agg, 
                       left_on = ['trip_id', 'stop_id', 'opd_date', 'rte'],
                       right_on = ['trip_id', 'stop_id', 'business_date', 'route_number'],
                       how = 'inner',
                       suffixes = ('_apc', '_orca'))
print('Rows, Features:', merged_data.shape)

In [ ]:
# check merged_data is unique
distinct_col = ['opd_date', 'trip_id', 'stop_id']
merge_dups = merged_data[merged_data[distinct_col].duplicated()]
print('Duplicate records:', merge_dups.shape[0])

In [ ]:
# roll up ons/counts to trip_id (drop stop_id)
trip_groupby = ['opd_date', 'day_of_week', 'trip_id', 'rte', 'is_rapidride', 'dir', 'route_number']
trip_sumover = ['ons', 'offs', 'ons_update'] + list(merged_data.columns)[-7:]
print(trip_sumover)
trip_agg = merged_data[trip_groupby + trip_sumover].groupby(trip_groupby).sum().reset_index()
print('Rows, Features:', trip_agg.shape)
trip_agg.head()

In [ ]:
# plot apc vs ORCA
plt.scatter(trip_agg['ons'], trip_agg['orca_total'])

In [ ]:
# find valid trips, where ocra <= apc and apc >= 0
trip_agg_sub0 = trip_agg[trip_agg['orca_total'] > trip_agg['ons']]
print('Rows where OCRA > APC for trip', trip_agg_sub0.shape[0])

trip_agg_sub1 = trip_agg[(trip_agg['ons'] >= 0) & (trip_agg['orca_total'] <= trip_agg['ons'])]
print('Rows where OCRA < APC and APC >= 0 for trip:', trip_agg_sub1.shape[0])

trip_agg_sub2 = trip_agg[(trip_agg['ons_update'] >= 0) & (trip_agg['orca_total'] <= trip_agg['ons_update'])]
print('Rows where OCRA < APC and APC >= 0 for trip (ons < 150 at every apc):', trip_agg_sub2.shape[0])

In [ ]:
# plot valid apc vs ORCA
plt.scatter(trip_agg_sub1['ons'], trip_agg_sub1['orca_total'])

In [ ]:
# plot valid apc vs ORCA (ons < 150 at every stop)
plt.scatter(trip_agg_sub1['ons_update'], trip_agg_sub1['orca_total'])

In [ ]:
# histogram of ORCA/APC ratio for trip/day
# trip_agg_sub1['ratio_0'] = trip_agg_sub1['orca_total']/trip_agg_sub1['ons']
trip_agg_sub2['ratio_1'] = trip_agg_sub2['orca_total']/trip_agg_sub2['ons_update']
print('Average ratio:', np.mean(trip_agg_sub2['ratio_1']))
plt.hist(trip_agg_sub2['ratio_1'], bins = 100)
plt.xlim(0, 1)

In [ ]:
# save datasets
orca_data.to_csv(filepath + 'data/ORCA_cleaned.csv')
print(orca_data.shape)
orca_agg.to_csv(filepath + 'data/ORCA_aggregate.csv')
print(orca_agg.shape)
avl_data.to_csv(filepath + 'data/APC_cleaned.csv')
print(avl_data.shape)
avl_agg.to_csv(filepath + 'data/APC_aggregate.csv')
print(avl_agg.shape)
trip_agg_sub2.to_csv(filepath + 'data/trip_rollup.csv')
print(trip_agg_sub2.shape)

In [ ]:
# get earliest trip time
trip_time = avl_data[['opd_date', 'trip_id', 'stop_datetime']].groupby(['opd_date', 'trip_id']).min().reset_index()

def hour_rounder(interval, t):
    if interval == .5:
        if t.minute > 30:
            x = t.replace(second=0, microsecond=0, minute=30, hour=t.hour).strftime('%H:%M:%S')
        else:
            x = t.replace(second=0, microsecond=0, minute=0, hour=t.hour).strftime('%H:%M:%S')
    else:
        y = t.hour
        if t.hour % interval != 0:
            y = t.hour - t.hour % interval
        x = t.replace(second=0, microsecond=0, minute=0, hour=y).strftime('%H:%M:%S')
    return x

trip_time['halfhr'] = [hour_rounder(.5, x) for x in pd.to_datetime(trip_time['stop_datetime'])]
trip_time['1hr'] = [hour_rounder(1, x) for x in pd.to_datetime(trip_time['stop_datetime'])]
trip_time['2hr'] = [hour_rounder(2, x) for x in pd.to_datetime(trip_time['stop_datetime'])]
trip_time['4hr'] = [hour_rounder(4, x) for x in pd.to_datetime(trip_time['stop_datetime'])]
trip_time['6hr'] = [hour_rounder(6, x) for x in pd.to_datetime(trip_time['stop_datetime'])]
trip_time.head()

In [ ]:
# make training sets
time_intervals = ['halfhr', '1hr', '2hr', '4hr', '6hr']

for val in time_intervals:
    set_data = trip_agg_sub2.drop(['route_number', 'ratio_1'], axis = 1)
    set_data = pd.merge(set_data , trip_time[['opd_date', 'trip_id', val]], 
                        left_on=['trip_id', 'opd_date'],
                        right_on=['trip_id', 'opd_date'],
                        how='inner',
                        suffixes=('_data', '_time'))
    set_data['ons'] = set_data['ons_update']
    set_data = set_data.drop(['trip_id', 'ons_update'], axis = 1).groupby(['opd_date', 'rte', 'dir', 'day_of_week', 'is_rapidride', val]).sum().reset_index()
    
    cols = list(set_data.columns)
    cols.remove('ons')
    X = set_data[cols]
    y = set_data['ons']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=1)
    
    print(np.sum(y))
#     print(set_data.head(5))
    print(val + "(train, test, val):", X_train.shape[0], X_test.shape[0], X_val.shape[0])
    X_train.to_csv(filepath + '/data/' + val + '/X_train.csv')
    X_test.to_csv(filepath + '/data/' + val + '/X_test.csv')
    X_val.to_csv(filepath + '/data/' + val + '/X_val.csv')
    y_train.to_csv(filepath + '/data/' + val + '/y_train.csv')
    y_test.to_csv(filepath + '/data/' + val + '/y_test.csv')
    y_val.to_csv(filepath + '/data/' + val + '/y_val.csv')

In [ ]:
# compare previous pipeline results with this
compare = pd.read_csv(filepath + '49_merged_at_stop_level.tsv.gz', sep='\t')
compare_dups = compare[['opd_date', 'trip_id', 'stop_id', 'ons', 'orca_total']][compare[['opd_date', 'trip_id', 'stop_id']].duplicated()]
print(compare_dups.shape)
compare_dups.head()

In [ ]:
# rte 49 from prior pipeline
compare[(compare['trip_id'] == 40684352) 
        & (compare['opd_date'] == '2019-03-01') 
        & (compare['stop_id'] == 1180)]

In [ ]:
# rte 49 from current pipeline
merged_data[(merged_data['trip_id'] == 40684352) 
            & (merged_data['opd_date'] == '2019-03-01') 
            & (merged_data['stop_id'] == 1180)]

In [ ]:
# rte 49 from raw ORCA
orca_data[(orca_data['business_date'] == '2019-03-01') 
          & (orca_data['trip_id'] == 40684352)
          & (orca_data['stop_id'] == 1180)]

In [ ]:
# rte 49 from raw APC
avl_data[(avl_data['opd_date'] == '2019-03-01')
         & (avl_data['trip_id'] == 40684352)
         & (avl_data['stop_id'] == 1180)]

In [ ]:
# total rte 49 APC and ORCA for 3/1/19 & 3/2/19 from prior pipeline
compare[['ons', 'orca_total']].sum()

In [ ]:
# total rte 49 APC and ORCA for 3/1/19 & 3/2/19 from this pipeline
trip_agg_sub2[['ons', 'orca_total']][((trip_agg_sub2['opd_date'] == '2019-03-01') 
                                      | (trip_agg_sub2['opd_date'] == '2019-03-02')) 
                                     &(trip_agg_sub2['rte'] ==49)].sum()